In [1]:
import numpy as np
from scipy import stats
import pymc3 as pm
import arviz as az
import matplotlib.pyplot as plt 
import pandas as pd
import seaborn as sns
az.style.use('arviz-darkgrid')
%matplotlib inline

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


# Loaded Coin Example - from algebra to PyMC3

You know someone has a coin which has a probability of 70% of coming up heads. They come up to you and say they will play a betting game with you, using what they say is a fair coin. You think the chances they are using the loaded coin is 60%.

The terms of the bet are that you're allowed to flip the coin 5 times, at which point you have to decide whether you want to play or not.

### Restating in statistical terms
First, think 'what do I want to know, what is my unknown here?' The answer is, 'Is this coin loaded'. Or stochastically, 'How certain am I this coin is loaded?' This is my hypothesis.

Denote this unknown variable as $\theta$. $\theta$ can take the values $\theta=1$ (the coin is loaded), or $\theta=0$ (the coin is fair).

I decide to represent $\theta$ as a bernoulli distribution, $\theta \sim \text{Bernoulli}(p)$, with unknown p. My initial guess at p is 0.6.

The results of the coin flipping exercise is denoted as $Y$. $Y$ is a binomial distribution. 

$$Y \sim \text{Binomial}(5, q)$$

$q$ takes the value 0.7 is the coin is loaded (i.e. if $\theta = 1$) and 0.5 if the coin is fair ($\theta = 0$)

### Solving Algebraically

I want to know what my posterior estimate for $\theta$ should be, if I perform experiment $Y$ and get $y$ heads. In other words, I want to know the function $f(\theta \mid y)$

Using Bayes's theorem:

$$f(\theta \mid y) = \frac{f(\theta)f(y \mid \theta)}{f(y)}$$

$$f(\theta) = 0.6^{\theta} 0.4^{1-\theta}$$

$$f\left(y \mid \theta\right) = {5 \choose y} q^y \left(1-q\right)^{5-y} = {5 \choose y} \left( 0.7^y \left(0.3\right)^{5-y} I_{\theta=1} + 0.5^5 I_{\theta=0} \right)$$

$$f(y) = {5 \choose y} 0.7^y 0.3^{5-y} 0.6 + {5 \choose y} 0.5^5 0.4$$ 

$$f(\theta \mid y) = \frac{0.6^{\theta} 0.4^{1-\theta} {5 \choose y} \left( 0.7^y \left(0.3\right)^{5-y} I_{\theta=1} + 0.5^5 I_{\theta=0} \right)}{{5 \choose y} 0.7^y 0.3^{5-y} 0.6 + {5 \choose y} 0.5^5 0.4} = \frac{0.6^{\theta} 0.4^{1-\theta} \left( 0.7^y \left(0.3\right)^{5-y} I_{\theta=1} + 0.5^5 I_{\theta=0} \right)}{0.7^y 0.3^{5-y} 0.6 + 0.5^5 0.4}$$


$$\frac{0.7^y 0.3^{5-y} 0.6 I_{\theta=1}}{0.7^y 0.3^{5-y} 0.6 + 0.5^5 0.4} + \frac{0.5^5 0.4}{0.7^y 0.3^{5-y} 0.6 + 0.5^5 0.4}$$
